In [136]:
from keras import Model
from keras.models import Sequential, Layer
from keras.layers import Dense, MaxPool2D, Dropout,MaxPool2D, Activation,BatchNormalization, InputLayer, Embedding, Flatten, Conv2D,concatenate, MaxPooling2D, Input, TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
import keras.utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.layers import Flatten, LSTM
import numpy as np
from keras import objectives
from __future__ import absolute_import

In [137]:
s1 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s2 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s3 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s4 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s5 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s6 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
data = np.array([s1,s2,s3,s4,s5,s6]).reshape(-1,10,64,64,3)
label = np.array([0,1,2,3,4,5]).reshape(-1)
print(s1.shape)
print(data.shape)
print(label.shape)
label = keras.utils.to_categorical(label, num_classes=None)
K.set_image_data_format('channels_last')
K.image_data_format()
K.set_learning_phase(1) 

(10, 64, 64, 3)
(6, 10, 64, 64, 3)
(6,)


In [138]:
def min_max_norm(cossim):
    min_m = K.min(cossim)
    max_m = K.max(cossim)
    normalized =[((x-min_m)/(max_m-min_m)) for x in cossim];
    print("success:: normalized")
    return normalized

In [141]:
def cossy(x_mat):
    nm_x = [K.l2_normalize(x_mat[i][:])  for i in range(10)]
    apex_vec = nm_x[-1]
    cossim = [K.dot(vec, apex_vec) for vec in nm_x]
    print("success:: cossim")
    return cossim

In [142]:
def cossimilasrity(x):
    cossim = cossy(x)
    normalized =min_max_norm(cossim)
    print("success:: cossimilarity")
    return normalized

In [143]:
def loss_func(y_true, y_pred):
    
    print("Begin:: loss_func")
    
    y_true = cossimilasrity(y_true)
    y_pred = cossimilasrity(y_pred)
    
    y_true = K.concatenate(y_true, axis=-1)
    y_pred = K.concatenate(y_pred, axis=-1)
    
    loss = objectives.mean_squared_error(y_true, y_pred)
    print("success:: loss_func")
    return loss

In [152]:
model = Sequential()
model.add(TimeDistributed(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal", name = 'c1'), input_shape=s1.shape))
model.add(TimeDistributed(Activation('relu')))
model.add(BatchNormalization())

model.add(TimeDistributed(Conv2D(64, (3,3),  padding='same', name = 'c2')))
model.add(TimeDistributed(Activation('relu')))
model.add(BatchNormalization())

model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Conv2D(32, (3,3),padding='same', name = 'c3')))
model.add(TimeDistributed(Activation('relu')))
model.add(BatchNormalization())

model.add(TimeDistributed(Conv2D(32, (3,3),padding='same', name = 'c4')))
model.add(TimeDistributed(Activation('relu')))
model.add(BatchNormalization())

model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for layer in model.layers[:18]:
    print("Deactivate:: " , layer)
    layer.trainable = False
    
for layer in model.layers[18:]:
    layer.trainable = True

model.add(LSTM(512, return_sequences=True, dropout=0.2, input_shape = (10,512),name='lstm_1' ))#E2 Objective Covered
model.compile(loss= loss_func, optimizer = 'adam', metrics=['accuracy'])

for layer in model.layers[:19]:
    print("Deactivate:: " , layer)
    layer.trainable = False
    
for layer in model.layers[19:]:
    layer.trainable = True

model.add(LSTM(6, return_sequences=False, dropout=0.2, input_shape = (None,10,512), name='lstm_2'))#E1 Objective Covered
model.add(Dense(6,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#'categorical_crossentropy'
model.summary()

Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c2595d68>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c2594f60>
Deactivate::  <keras.layers.normalization.BatchNormalization object at 0x1c2595fd0>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c25d69e8>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c25ffe10>
Deactivate::  <keras.layers.normalization.BatchNormalization object at 0x1c25eeeb8>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c2690a58>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c26b91d0>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c270eb38>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c58b1240>
Deactivate::  <keras.layers.normalization.BatchNormalization object at 0x1c589eac8>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0x1c58e9fd0>
Deactivate::  <keras.layers.wrappers.TimeDistributed object at 0

In [153]:
model.fit(data, label, batch_size=1, epochs = 10)
# score, acc = model.evaluate(data, label)
# print('Test score:', score)
# print('Test accuracy:', acc)

Epoch 1/10
6/6 [==============================] - 14s 2s/step - loss: 2.0725 - acc: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 6s 1s/step - loss: 1.7781 - acc: 0.3333
Epoch 3/10
6/6 [==============================] - 6s 1s/step - loss: 1.7645 - acc: 0.3333
Epoch 4/10
6/6 [==============================] - 8s 1s/step - loss: 1.8648 - acc: 0.3333
Epoch 5/10
6/6 [==============================] - 8s 1s/step - loss: 1.7754 - acc: 0.1667
Epoch 6/10
6/6 [==============================] - 7s 1s/step - loss: 1.7398 - acc: 0.3333
Epoch 7/10
6/6 [==============================] - 6s 1s/step - loss: 1.9395 - acc: 0.1667
Epoch 8/10
6/6 [==============================] - 6s 1s/step - loss: 1.8356 - acc: 0.3333
Epoch 9/10
6/6 [==============================] - 6s 972ms/step - loss: 1.8015 - acc: 0.3333
Epoch 10/10
6/6 [==============================] - 6s 1s/step - loss: 1.7896 - acc: 0.1667


In [28]:
#cnn mian seperate import
cnnmodel = Sequential()
cnnmodel.add(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal", 
                 input_shape=(64, 64, 3)))
#cnnmodel.add(BatchNormalization())
cnnmodel.add(Activation("relu"))

cnnmodel.add(Conv2D(64, 4))
#cnnmodel.add(BatchNormalization())
cnnmodel.add(Activation("relu"))
cnnmodel.add(MaxPool2D(pool_size=(2, 2), strides=2))
cnnmodel.add(Dropout(0.5))

cnnmodel.add(Conv2D(32, 3))
#cnnmodel.add(BatchNormalization())
cnnmodel.add(Activation("relu"))

cnnmodel.add(Conv2D(32, 3))
cnnmodel.add(keras.layers.normalization.BatchNormalization())#BatchNormalization())
cnnmodel.add(Activation("relu"))

cnnmodel.add(MaxPool2D(pool_size=(2, 2), strides=2))
cnnmodel.add(Dropout(0.5))

cnnmodel.add(Flatten())
cnnmodel.add(Dense(512))
cnnmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnnmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 60, 60, 64)        4864      
_________________________________________________________________
activation_5 (Activation)    (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 57, 57, 64)        65600     
_________________________________________________________________
activation_6 (Activation)    (None, 57, 57, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 26, 26, 32)        18464     
__________

In [14]:
## approach 2
model = Sequential()
sequence = Input(shape=[None, 64, 64, 3])
model.add(TimeDistributed(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal"), input_shape=(1000, 64, 64, 3)))
model.add(TimeDistributed(Conv2D(32, (3,3),  padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(BatchNormalization())
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=True, dropout=0.5))
model.add(Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 1000, 60, 60, 64)  4864      
_________________________________________________________________
time_distributed_9 (TimeDist (None, 1000, 58, 58, 32)  18464     
_________________________________________________________________
time_distributed_10 (TimeDis (None, 1000, 29, 29, 32)  0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, 1000, 29, 29, 64)  18496     
_________________________________________________________________
time_distributed_12 (TimeDis (None, 1000, 29, 29, 64)  36928     
_________________________________________________________________
time_distributed_13 (TimeDis (None, 1000, 14, 14, 64)  0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 1000, 14, 14, 128) 73856     
__________